In [1]:
import re
import time

import ee
import geemap
import geopandas as gpd
from google.cloud import storage

ee.Authenticate()

ee.Initialize(project="ee-ciut")

In [2]:
# Constants
BUCKET_NAME = "ciut-tablero"
GOOGLE_CLOUD_PROJECT = "ee-ciut"

In [3]:
import sys
import os
sys.path.append(os.path.abspath('..'))  # Add the parent directory to Python path
from utils.etl import load_exported_datasets

In [4]:
datasets = load_exported_datasets()

Successfully loaded population dataset
Successfully loaded impervious dataset
Successfully loaded wetlands dataset
Successfully loaded nighttime_lights dataset
Successfully loaded vulnerability dataset


In [5]:
datasets

{'population': <ee.image.Image at 0x741e69bd0320>,
 'impervious': <ee.image.Image at 0x741e69d6df10>,
 'wetlands': <ee.image.Image at 0x741e69d43ce0>,
 'nighttime_lights': <ee.image.Image at 0x741e69d6e540>,
 'vulnerability': <ee.image.Image at 0x741e69d6ddc0>}

In [6]:
from utils.etl import get_municipalities

In [7]:
municipalities = get_municipalities()
basins_gdf = gpd.read_file(
        "/home/nissim/Documents/dev/ciut-tablero/data/dipsoh_cuencas.geojson"
    )
basins_ee = geemap.geopandas_to_ee(basins_gdf)
intersecting_basins = basins_ee.filterBounds(municipalities.geometry())
aoi = intersecting_basins.geometry()

In [10]:
gisa = datasets["impervious"]
population = datasets["population"]
wetlands = datasets["wetlands"]
nighttime_lights = datasets["nighttime_lights"]
vulnerability = datasets["vulnerability"]

In [11]:
# Create map and center on AOI
Map = geemap.Map(basemap="CartoDB.Positron")
Map.centerObject(aoi, 9)

Map.addLayer(gisa, {}, "GISA 1972–2019")

EEException: Failed to read object "gs://ciut-tablero/gisa_2019.tif". No such object: ciut-tablero/gisa_2019.tif

In [ ]:
Map.addLayer(
    population,
    {
        "bands": ["population"],
        "min": 0.0,
        "max": 50.0,
        "palette": ["24126c", "1fff4f", "d4ff50"],
    },
    "WorldPop Population 100m",
)

In [9]:




ntl_vis = {
    "opacity": 1,
    "bands": ["avg_rad"],
    "min": 1,
    "max": 32,
    "palette": ["584d9f", "9c79c1", "c98cbe", "f2d192", "e2ee82"],
}
Map.addLayer(nighttime_lights, ntl_vis, "NTL VIIRS")

wetlands_vis = {
    "min": 180,
    "max": 187,
    "palette": [
        "#CCCCCC",
        "#0000FF",
        "#006400",
        "#00FF00",
        "#00FFFF",
        "#CC99FF",
        "#556B2F",
        "#FFFF99",
        "#D2B48C",
    ],
}
Map.addLayer(wetlands.mask(wetlands.neq(0)), wetlands_vis, "GWL FCS30 Wetlands")

Map.addLayer(
    intersecting_basins.style(
        **{"color": "black", "fillColor": "00000000", "width": 1}
    ),
    {},
    "Intersecting Basins",
)

Map

EEException: Failed to read object "gs://ciut-tablero/gisa_2019.tif". No such object: ciut-tablero/gisa_2019.tif